In [3]:
from typing import List
import os

import pandas as pd
import torch

In [ ]:
training_years = [2015, 2016, 2017, 2018]
dev_years = [2019]
test_years = [2020]

In [2]:
class Data(torch.utils.data.Dataset):
    """Data class holds our data and allows easy access.
    Must implement `__len__()` and `__getitem__()`
    """
    
    def __init__(self, years: List[int], domain: str):
        self.years = years
        self.lengths = []
        for y in self.years:
            path = os.path.join('data', domain, f'{y}-nba-schedule.csv')
            df = pd.read_csv(index_col=0)
            self.lengths.append( len(df) )
            
    def __len__(self):
        """The length of our data is how many games there are."""
        return sum(self.lengths)
        
    def __getitem__(self, key: int):
        """We only support int indices right now so we don't have to open 
        multiple files.
        """
        if not isinstance(key, int):
            msg = f'Data can only be indexed with a scalar, {key} attempted'
            raise TypeError(msg)
            
        
        